In [ ]:
import numpy as np
import faiss
from functions import create_color_histogram, create_vit_embedding, get_glcm_features, extract_hog_features, extract_cnn_features
import pandas as pd
import cv2
import os

Cargamos los índices

In [2]:
color_histograms_index = faiss.read_index("database/color_histograms.index") 
texture_histograms_index = faiss.read_index("database/texture_histograms.index")
hog_index = faiss.read_index("database/hog.index")
cnn_index = faiss.read_index("database/cnn.index")
embeddings_index = faiss.read_index("database/vit.index")

Definimos la función de evaluación

In [3]:
def precision_at_k(query, k, index, ground_truth):
    faiss.normalize_L2(query)
    _, I = index.search(query, k)
    return len(set(I[0]) & ground_truth) / k

Cargamos la base de datos de test

In [4]:
test_db = pd.read_csv("database/db_test.csv", index_col=0)

Definimos ground_truth_dic (son las etiquetas que servirán para evaluar el sistema)

In [5]:
ground_truth_dic = {"Coast": set(range(100)), 
                    "Desert": set(range(100, 200)), 
                    "Forest": set(range(200, 300)), 
                    "Glacier": set(range(300, 400)), 
                    "Mountain": set(range(400, 500))}

Definimos la function que evaluará cada método en cada clase

In [6]:
def evaluate_precision_at_k(func, index, test_db, k):
    precisions_by_class = {label: [] for label in ground_truth_dic.keys()}
    for test_image, label in test_db[['image', 'label']].values:
        path = "images/" + test_image
        img = cv2.imread(path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        query = func(img_rgb).reshape(1, -1)
        ground_truth = ground_truth_dic[label]
        precisions_by_class[label].append(precision_at_k(query, k, index, ground_truth))
    return precisions_by_class

Para evitar kernel crashes...

In [7]:
# Evitar conflicto de OpenMP
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Desactiva GPU en TensorFlow

Evaluamos cada método en cada clase usando la precision@k

In [8]:
methods = ["Color Histogram", "Texture Histogram", "HOG", "CNN", "VIT"]
funcs = [create_color_histogram, get_glcm_features, extract_hog_features, extract_cnn_features, create_vit_embedding]
indexes = [color_histograms_index, texture_histograms_index, hog_index, cnn_index, embeddings_index]
results = {}

for i in range(5):
    method = methods[i]
    func = funcs[i]
    index = indexes[i]
    precision_k = evaluate_precision_at_k(func, index, test_db, 5)
    results[method] = precision_k

1/1 [==============================] - 0s 176ms/step


Organizamos y vemos resultados

In [9]:
df = pd.DataFrame(results)
df = df.map(np.mean)
df.loc['MEAN'] = df.mean()
df['MEAN'] = df.mean(axis=1)
df.loc['MEAN', 'MEAN'] = ""
df

C:\Users\javip\AppData\Local\Temp\ipykernel_20332\3635353071.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc['MEAN', 'MEAN'] = ""


,Color Histogram,Texture Histogram,HOG,CNN,VIT,MEAN
Coast,0.340,0.320,0.140,0.460,0.940,0.44
Desert,0.280,0.280,0.280,0.380,1.000,0.444
Forest,0.460,0.540,0.840,0.640,0.420,0.58
Glacier,0.660,0.380,0.000,0.300,0.980,0.464
Mountain,0.240,0.200,0.120,0.300,0.840,0.34
MEAN,0.396,0.344,0.276,0.416,0.836,
